In [1]:
import os

import pandas as pd
import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer#, IterativeImputer

from sklearn.feature_selection import SelectKBest, mutual_info_regression, f_regression## exp needed!

from sklearn.ensemble import IsolationForest## exp needed!

from sklearn.preprocessing import StandardScaler, QuantileTransformer## exp needed!

from sklearn.svm import SVR## exp needed!

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV## exp needed!

from sklearn.metrics import r2_score

## 0. Parameters

### 0.1 Global Params

In [2]:
root_path = './'
X_train_path = 'X_train.csv'
X_test_path = 'X_test.csv'
y_train_path = 'y_train.csv'
y_test_path = 'y_test_yutong_v3.csv'

val_size = 50
random_state = 30

### 0.2 Feature Selection

In [3]:
num_features = 150## about 200 real features

#### 0.2.1 Isolation Forest

In [4]:
n_estimators = 100
contamination = 'auto'## not tuned yet
max_features = 1.0## not tuned yet

### 0.3 Regresion
#### 0.3.1 SVR: Grid

In [5]:
svr_param_grid = {
    'C': [1, 10, 100, 1000], 
    'gamma': [1, 0.1, 0.01, 0.001]}## good start, as a ref to random search

### 0.4 Model Selection

In [6]:
num_KFold = 10## better than 5-fold

## 1. Data Loading

In [7]:
def data_raw(root_path, data_path):
    return pd.read_csv(os.path.join(root_path, data_path)).values[:,1:]

X_train_raw = data_raw(root_path, X_train_path)
X_test_raw = data_raw(root_path, X_test_path)
y_train = data_raw(root_path, y_train_path)

## 2. Data Wrangling & Preprocessing

### 2.1 Imputation Methods

In [8]:
def median_imp(X_raw):
    imp_med = SimpleImputer(missing_values=np.nan, strategy='median')
    return imp_med.fit_transform(X_raw)

def knn_imp(X_raw):##to-do, still a comp intensive methods and suffers from curse of dimensionality and outliers
    pass

def mice_imp(X_raw):#use after fea sel! this method is comp expensive, and is an unstable implementation based on docs
    imp_mice = IterativeImputer(missing_values=np.nan, initial_strategy='median')
    return imp_mice.fit_transform(X_raw)

> Remark: The median imputation is too naive to be proper in our situation. The best ones, I believe, are regression/interpolation methods like mice but mice suffers from high dims. Also, there may be no sklearn implementation for other possible methods. We may consider doing feature selection b4 this step but beware there can be problems of preprocessing orders that Fabian mentioned.

In [9]:
X_train_raw = median_imp(X_train_raw)
X_test_raw = median_imp(X_test_raw)

### 2.2 Feature Selection Methods

In [10]:
def feat_sel(X_raw, y, score_func, num_features):
    return SelectKBest(score_func = score_func, k=num_features).fit(X_raw, y.ravel())

In [11]:
selector = feat_sel(X_train_raw, y_train, f_regression, num_features)

X_train_raw = selector.transform(X_train_raw)
X_test = selector.transform(X_test_raw)

/home/leona/anaconda3/envs/proj0/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:284: RuntimeWarning: invalid value encountered in sqrt
  X_norms = np.sqrt(row_norms(X.T, squared=True) - n_samples * X_means ** 2)
/home/leona/anaconda3/envs/proj0/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


> Remark: most recent exps show that compared to mutual_info, f_regression are insensitive (if chosen properly) to #features and #estimators for isolation forests, and produce better results. We can check the scores to determine #features to choose. My lucky range falls on 100-200, which is similar to the actual #features mentioned during the exercise session.

> there are even more complex feature sel criteria like cond infomax which does more than f_reg and mi. We need extra lib for this if we wanna try

### 2.3 Outlier Detection Methods

In [12]:
def train_val_split(X_train, y_train, val_size):
    return train_test_split(X_train, y_train, test_size=val_size, random_state=random_state)

def iforest(X_raw, y, n_estimators):## param control
    iso = IsolationForest(n_estimators = n_estimators, random_state=random_state).fit_predict(X_raw)
    return X_raw[np.where(iso==1)], y[np.where(iso==1)]

> Remark: isolation forest is an unsupervised, log-time outlier detection method which is extremely suitable for high dim cases so I prefer it here. We can also try out dbscan, lof, rrcf since we've done feature selection and free from high dim (but 200 features is still not sth easy....

In [13]:
X_train_raw, X_val, y_train, y_val = train_val_split(X_train_raw, y_train, val_size)

X_train, y_train = iforest(X_train_raw, y_train, n_estimators)

> Remark: sth nuanced here: do val set need outlier detection? my answer is no. Val set should mimic the behavior of the test set which CAN contain outliers, and we do not expect our model to perform well on outliers anyway. But current implementation of gridsearch are using cleaned val sets, and i am not sure if this introduces problems...

### 2.4 Scaling

In [14]:
X_train = StandardScaler().fit_transform(X_train)
X_val = StandardScaler().fit_transform(X_val)
X_test = StandardScaler().fit_transform(X_test)

> Remark: Scaling is important for svr. Besides std scaler we can also try out QuantileTransformer which can deal with outliers, but I believe scaling would not affect much.

## 3. Regression & Param Tuning

In [15]:
svr = SVR()
reg = GridSearchCV(svr, svr_param_grid, scoring='r2', cv=num_KFold).fit(X_train, y_train.ravel())

> Remark: I like SVR because it is non-linear and has infinite representiveness if params are fine-tuned as far as I know. Also diff kernel funcs can produce similar results so all we have to do is to tune C and gamma for rbf kernel. There are a bunch of other reg methods to explore but I prefer to stick to SVR at the moment.

> for model sel we can also try random search here

In [16]:
reg.best_estimator_

SVR(C=100, gamma=0.01)

## 4. Evaluation

In [17]:
y_val_pred = reg.predict(X_val)

score = r2_score(y_val, y_val_pred)
print(score)

0.6913236293776486


## 5. Results

> Other to-dos: order of preprocessing, ensemble, pipeline, ...